In [1]:
import os
path = '/Users/danmontesi/Desktop/recsys2019'
os.chdir(path)
print(os.getcwd())

/Users/danmontesi/Desktop/recsys2019


In [2]:
import data
import pandas as pd
import numpy as np

In [3]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from xgboost import XGBClassifier

In [4]:
from tqdm.auto import tqdm  # for notebooks
tqdm.pandas()

In [5]:
def existsIndex(df, idx):
    try:
        df.loc[idx]
        return True
    except KeyError:
        return False

In [6]:
def RepresentsInt(s):
    try: 
        int(s)
        return True
    except TypeError:
        return False
    except ValueError:
        return False

In [7]:
def getNumOfAction(df, action_type):
    return len(df[df["action_type"] == action_type])

In [8]:
dataset = data.train_df("full")


/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [9]:
df = dataset.copy()

In [125]:
def func(x):
    
    y = x[x['action_type'] == 'clickout item']
    
    #keep only sessions until last clickout

    clk = y.tail(1)
    
    if existsIndex(x, clk.index - 1):
        #exists numeric reference before
        if RepresentsInt(x.loc[clk.index - 1]['reference']):

            x = x[ x["step"] < int(clk["step"])]
            
            impressions = list(map(int, clk['impressions'].values[0].split('|')))
    
            num_ref_row = x.loc[clk.index - 1]
            
            x_only_numeric_ref = x[pd.to_numeric(x['reference'], errors='coerce').notnull()]
            
            interacted = list(x_only_numeric_ref.reference)
            
            # feature computation
            info_perc = getNumOfAction(x, "interaction item info") / len(x_only_numeric_ref)
            
            rating_perc = getNumOfAction(x, "interaction item rating") / len(x_only_numeric_ref)
            
            deals_perc = getNumOfAction(x, "interaction item deals") / len(x_only_numeric_ref)
            
            image_perc = getNumOfAction(x, "interaction item image") / len(x_only_numeric_ref)
            
            clickout_count = getNumOfAction(x, "clickout item")
            
            poi_count = getNumOfAction(x, "search for poi")
            
            search_destination_count = getNumOfAction(x, "search for destination")
            
            search_item_count = getNumOfAction(x, "search for item")
            
            filter_sel_count = getNumOfAction(x, "filter selection")
            
            sort_change_count = getNumOfAction(x, "change of sort order")
            
            # time elapsed
            time_elapsed = int(clk['timestamp']) - int(num_ref_row['timestamp'])
            
            clicked = num_ref_row['reference'].values[0]

            appear = 0
            for el in interacted[1:]:
                if el == clicked:
                    appear +=1
                    
            label = None
            if appear>0:
                label = 1
            else:
                label = 0
            
            return pd.DataFrame({'time_elapsed': [time_elapsed], "info_perc": [info_perc],
                                 "rating_perc": [rating_perc], "deals_perc": [deals_perc], "image_perc": [image_perc], "clickout_count": [clickout_count],
                                 "poi_count": [poi_count], "search_destination_count": [search_destination_count], "search_item_count": [search_item_count], 
                                 "filter_sel_count": [filter_sel_count], "sort_change_count": [sort_change_count],  
                                 'label': [label]})

In [127]:
dataset = df[:1000000].groupby(['user_id', 'session_id']).progress_apply(func)

HBox(children=(IntProgress(value=0, max=57238), HTML(value='')))

In [128]:
dataset.head(200)

,,,clickout_count,deals_perc,filter_sel_count,image_perc,info_perc,label,poi_count,rating_perc,search_destination_count,search_item_count,sort_change_count,time_elapsed
user_id,session_id,,,,,,,,,,,,,
0004IOZI7CKF,0146f7cb014ba,0,2,0.000000,0,0.000000,0.333333,1,0,0.000000,1,0,0,27
002BISXP1U1Q,8cd721ffb8e03,0,0,1.000000,0,0.000000,0.000000,0,0,0.000000,0,0,0,43
002IXCPKK20W,dcdc75a82f142,0,0,0.000000,0,0.000000,0.000000,0,0,0.000000,0,1,0,56
002O8H8GYKSQ,ab252cd2bafd3,0,1,0.000000,0,0.000000,0.000000,0,0,0.000000,0,0,0,74
003ZD97CWBYE,f833a387fa0c9,0,1,0.333333,0,0.000000,0.000000,1,0,0.333333,0,0,0,39
004STL2FQU99,24ab1f448fa9c,0,1,0.000000,1,0.000000,0.500000,1,0,0.000000,7,0,2,14
006MM63UET1V,e5d37102d5f21,0,0,0.000000,0,1.000000,0.000000,1,0,0.000000,0,0,0,1
006OQ8Q1YFFY,cbe7976eabb3f,0,3,0.000000,0,0.400000,0.000000,1,0,0.000000,0,0,0,375
00941AJZU07K,a337682a6f448,0,5,0.000000,6,0.000000,0.000000,1,0,0.583333,0,0,0,44


In [130]:
print(len(dataset[ dataset["label"] == 0]))
    

7063


In [131]:
X, y = dataset.iloc[:,list(range(0,5)) + list(range(6,12))],dataset.iloc[:,5]

In [132]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [133]:
import xgboost as xgb

In [134]:
xg_reg = xgb.XGBClassifier(objective ='binary:logistic')#, colsample_bytree = 0.3, learning_rate = 0.1,
                #max_depth = 5, alpha = 10, n_estimators = 10)

In [135]:
xg_reg.fit(X_train,y_train)

preds = xg_reg.predict(X_test)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [136]:
target_names = ['clk_diff_ref', 'clk_eq_ref']
print(classification_report(y_test, preds, target_names=target_names))

              precision    recall  f1-score   support

clk_diff_ref       0.97      0.99      0.98      1391
  clk_eq_ref       1.00      0.99      0.99      4767

 avg / total       0.99      0.99      0.99      6158



In [137]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [138]:
accuracy = accuracy_score(y_test, [0]*len(preds))
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 22.59%


In [139]:
accuracy = accuracy_score(y_test, [1]*len(preds))
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 77.41%


In [140]:
accuracy = accuracy_score(y_test,  preds)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 99.06%
